## Complete Real-Time Algorithm

### Used Variables

\begin{equation}
h, p, \beta, x, d, \alpha, A, y, i, j, l, w, b, m, s, v
\end{equation}

### Necessary Definitions

---

$\hat{h} \in \mathbb{R}$ := The measured length (in pixels) of a honeybee in the video, pre-sampled and averaged across $5$ different time points

---

$\hat{p} \in (0, 1) $ := The pre-fit probability threshold for the logistic regression model

---

$\hat{\beta}_0 \in \mathbb{R}$ := The pre-fit logistic regression intercept

$\hat{\beta}_1 \in \mathbb{R}$ := The pre-fit logistic regression coefficient for mean distance

$\hat{\beta}_2 \in \mathbb{R}$ := The pre-fit logistic regression coefficient for mean vertex angle

$\hat{\beta}_3 \in \mathbb{R}$ := The pre-fit logistic regression coefficient for their interaction

$\hat{\beta} = \begin{bmatrix} \hat{\beta}_0 & \hat{\beta}_1 & \hat{\beta}_2 & \hat{\beta}_3 \end{bmatrix}^T \in \mathbb{R}^4$

---

$y_k \in \left\{ 0, 1 \right\}$ := The actual classification of a honeybee's location at time step $t = k$, where:
* $y_k = 1$ signifies that the bee is on the waggle *run*, and 
* $y_k = 0$ signifies that the bee is on the *return* phase

---

$w(l)$, $b_-(l)$, $b_+(l)$, and $m(l)$ are defined (and can be computed prior to program execution for $l = 1, \dots, 6$) as

\begin{align}
w(l) & = 3l + 2 \\
b_-(l) & = 2l \\
b_+(l) & = l + 1 \\
m(l) & = l - 1 \\
\end{align}

---

#### `getcurrentloc()`

The function `getcurrentloc()` returns the current (meaning, at time step $t = k$) location of the honeybee. The location of the honeybee at time step $t = k$ is given by

\begin{equation}
\overrightarrow{x_k} = \begin{bmatrix} x^{(1)}_k \\ x^{(2)}_k \end{bmatrix} \in \mathbb{R}^2,
\end{equation}

where $x^{(1)}_k$ is the honeybee's *horizontal* coordinate (in pixels) at time $k$ and $x^{(2)}_k$ is the honeybee's *vertical* coordinate (in pixels) at time $k$.

### Algorithm (Expressed Mathematically)

\begin{align*}
& \text{while} \left( \left( \overrightarrow{x}_k \leftarrow \text{getcurrentloc()} \right) \neq \text{NULL} \right): \\
\\
& \;\;\;\;\; \tilde{x}_k = \left\{ \overrightarrow{x}_{k - 2}, \, \overrightarrow{x}_{k - 1}, \, \overrightarrow{x}_k \right\} & \text{# Location Buffer, as Array} \\
\\
& \;\;\;\;\; d_k = \frac{1}{\hat{h}}\cdot\left\Vert \overrightarrow{x}_k - \overrightarrow{x}_{k - 1} \right\Vert_2 \in (0, \approx 1] & \text{# Distance (normalized by bee length)} \\
& \;\;\;\;\; \tilde{d}_k = \left\{ d_{k - 5}, d_{k - 4}, d_{k - 3}, d_{k - 2}, d_{k - 1}, d_k \right\} & \text{# Distance Buffer, as Array} \\
\\
& \;\;\;\;\; \alpha_{k - 1} = \frac{1}{\pi}\cdot\arccos{\left(\frac{\frac{1}{\hat{h}}\cdot\left\Vert \overrightarrow{x}_k - \overrightarrow{x}_{k - 2} \right\Vert_2^2 - d_{k}^2 - d_{k - 1}^2}{2 \cdot d_{k} \cdot d_{k - 1}}\right)} \in [0, 1] & \text{# Suppl. Vertex Angle (computed w/ the Law of Cosines, normalized by $\pi$)} \\
& \;\;\;\;\; \tilde{\alpha}_k = \left\{ \alpha_{k - 5}, \alpha_{k - 4}, \alpha_{k - 3}, \alpha_{k - 2}, \alpha_{k - 1} \right\} & \text{# Angle Buffer, as Array} \\
\\
& \;\;\;\;\; \bar{d}_{k - 3} = \frac{1}{6}\sum_{i = k - 5}^k{d_i} & \text{# Mean Distance} \\
& \;\;\;\;\; \bar{\alpha}_{k - 3} = \frac{1}{5}\sum_{i = k - 5}^{k - 1}{\alpha_i} & \text{# Mean Angle} \\
\\
& \;\;\;\;\; \overrightarrow{A}_{k - 3} = \begin{bmatrix} 1 & \bar{d}_{k - 3} & \bar{\alpha}_{k - 3} & \bar{d}_{k - 3} \cdot \bar{\alpha}_{k - 3} \end{bmatrix}^T \in \mathbb{R}^4 \\
\\
& \;\;\;\;\; \hat{y}_{k - 3} = \left\lfloor \left(\frac{1}{1 + e^{-\overrightarrow{A}_{k - 3}^T\hat{\beta}}}\right) + \left(1 - \hat{p}\right)\right\rfloor \in \left\{ 0, 1 \right\} & \text{# Predicted Class using pre-fit Logistic Regression Model} \\
\\
& \;\;\;\;\; \tilde{y}_k = \left\{ \hat{y}_{k - 21}, \dots, \hat{y}_{k - 3} \right\} & \text{# Prediction Buffer} \\
\\
& \;\;\;\;\; i = k - 3 \\
& \;\;\;\;\; \text{for } l = 1, \dots, 4: \\ 
& \;\;\;\;\; \;\;\;\;\; i = i - b_+(l) \\
& \;\;\;\;\; \;\;\;\;\; \text{if } l = 4: \\
& \;\;\;\;\; \;\;\;\;\; \;\;\;\;\; i = i + b_+(3) \\
& \;\;\;\;\; \;\;\;\;\; s_i = \sum_{j = i - m(l)}^i{\hat{y}_j} \\
& \;\;\;\;\; \;\;\;\;\; \text{if } \left(\left[\frac{s_i}{l} \text{ mod } 1\right] == 0\right) \text{ and } \left(\hat{y}_{i - m} \neq \hat{y}_{i - m - 1}\right) \text{ and } \left(\hat{y}_i \neq \hat{y}_{i + 1}\right): \\
& \;\;\;\;\; \;\;\;\;\; \;\;\;\;\; v_i = \left\lfloor \frac{\left[\sum_{j = i - b_-}^{i + b_+}{\hat{y}_j}\right] - s_i}{w(l) - l } + \frac{1}{2} \right\rfloor \\
& \;\;\;\;\; \;\;\;\;\; \;\;\;\;\; \hat{y}_{i - j} = v_i, \text{for } j = 0, 1, \dots, m(l) \\
\\
& \text{# At this point, all pts up through $y_{k - 13}$ have been smoothed} \\
\end{align*}

### Algorithm (Expressed in Code)

In [7]:
dance = WaggleDance(locs_10fps_dir, locs_10fps_files[0])
logreg_model = logreg_10fps_model

min_run_length = 4
x_buffer = [0] * 4 # {k-3, k-2, k-1, k } # TODO - Extend through k-13
d_buffer = [0] * 6 # { k-5, k-4, k-3, k-2, k-1, k }
alpha_buffer = [0] * 5 # { k-5, k-4, k-3, k-2, k-1 }
y_buffer = [0] * 19 # { k-21, ..., k-3 }
curr_run_buffer = []

k = 0
x_k = dance.getcurrentloc()
while (len(x_k) != 0):
#     print('Time Step: k = {:d}'.format(k))
    x_buffer.pop(0)
    x_buffer.append(x_k)
    
    if k >= 1:
        d_k = euclid_dist(x_k, x_buffer[2])
        d_buffer.pop(0)
        d_buffer.append(d_k)
        
    if k >= 2: 
        alpha_kminus1 = suppl_vertex_angle(x_k, x_buffer[1], d_k, d_buffer[4])
        alpha_buffer.pop(0)
        alpha_buffer.append(alpha_kminus1)

    if k >= 6:
        d_bar_kminus3 = np.mean(d_buffer) / dance.h
        alpha_bar_kminus3 = np.mean(alpha_buffer) / np.pi

        A_kminus3 = np.array([1, d_bar_kminus3, alpha_bar_kminus3, d_bar_kminus3*alpha_bar_kminus3])

        y_hat_kminus3 = logreg_model.predict(A_kminus3)
        y_buffer.pop(0) # TODO - What do I do with the y_hat_minus22?
        y_buffer.append(y_hat_kminus3)

        print('Prediction at time k={:d}: {:d}'.format(k - 3, y_hat_kminus3))
        
    
    ###### Smoothing Algorithm

    # if k >= 10: # Boundary for l = 1 (Single)
    #     smooth_window(y_buffer, l=1, i=16) # i = 16 : Idx of y_hat_kminus8
    #
    # if k >= 15: # Boundary for l = 2 (Double)
    #     smooth_window(y_buffer, l=2, i=13) # i = 13 : Idx of y_hat_kminus8
    #
    # if k >= 21: # Boundary for l = 3 (Triple)
    #     smooth_window(y_buffer, l=3, i=9) # i = 9 : Idx of y_hat_kminus12
    #
    # if k >= 24: # Boundary for l = 4 (Quadruple)
    #     smooth_window(y_buffer, l=4, i=8) # i = 8 : Idx of y_hat_kminus13
    #
    #     y_ready = y_buffer[:8] # Points y_hat_kminus21 to y_hat_kminus16

    if k >= 6:
        if y_buffer[-1] == 1: #check prediction for xkminus_3 (latest prediction)
            curr_run_buffer.append(x_buffer[0]) #append xkminus_3
        else:
            if len(curr_run_buffer) <= min_run_length:
                curr_run_buffer.clear()
            else:
                slope, intercept = tls([i[0] for i in curr_run_buffer], [-i[1] for i in curr_run_buffer]) #Negative on the y values to reflect across x-axis
                theta = np.arctan(1/slope) #1/slope for cotangent
                run_length = len(curr_run_buffer)/10 #Length of run in frames divided by frames per second
                print("Run Angle (Radians) = {:.4f} Run Length (Seconds) = {:.4f}".format(theta, run_length))
                curr_run_buffer.clear()
          
    k += 1
    x_k = dance.getcurrentloc()

Prediction at time k=3: 1
Prediction at time k=4: 1
Prediction at time k=5: 1
Prediction at time k=6: 1
Prediction at time k=7: 0
Prediction at time k=8: 0
Prediction at time k=9: 0
Prediction at time k=10: 0
Prediction at time k=11: 0
Prediction at time k=12: 1
Prediction at time k=13: 1
Prediction at time k=14: 1
Prediction at time k=15: 1
Prediction at time k=16: 1
Prediction at time k=17: 1
Prediction at time k=18: 1
Prediction at time k=19: 0
Run Angle (Radians) = 0.7238 Run Length (Seconds) = 0.7000
Prediction at time k=20: 1
Prediction at time k=21: 1
Prediction at time k=22: 1
Prediction at time k=23: 0
Prediction at time k=24: 0
Prediction at time k=25: 0
Prediction at time k=26: 0
Prediction at time k=27: 0
Prediction at time k=28: 0
Prediction at time k=29: 0
Prediction at time k=30: 0
Prediction at time k=31: 0
Prediction at time k=32: 0
Prediction at time k=33: 0
Prediction at time k=34: 0
Prediction at time k=35: 0
Prediction at time k=36: 0
Prediction at time k=37: 0
Pre

In [127]:
b_hi[1], b_hi[2], b_hi[3], b_hi[4]

(2, 3, 4, 5)

## Necessary Code

(Not central to the algorithm itself, but makes the algorithm appear cleaner. Needs to be run first.)

In [2]:
# Libraries
import numpy as np
import pandas as pd
import scipy.odr as odr
import os

# Constants
data_dir = '../Data/'
metadata_file = '../Data/Dances_Metadata.csv'
metadata = pd.read_csv(metadata_file)

locs_10fps_dir = data_dir + 'Locations/10FPS/'
locs_10fps_files = os.listdir(locs_10fps_dir)
locs_10fps_files.remove('.DS_Store')
locs_10fps_files.sort()

locs_orig_dir = data_dir + 'Locations/Original/'
locs_orig_files = os.listdir(locs_orig_dir)
locs_orig_files.remove('.DS_Store')
locs_orig_files.sort()

all_models = pd.read_csv(data_dir + 'ClassificationModels/LogisticRegression.csv')

# Window Parameters (Constants)
w = {l:(3 * l + 2) for l in range(1, 4 + 1)}
b_lo = {l:(2 * l) for l in range(1, 4 + 1)}
b_hi = {l:(l + 1) for l in range(1, 4 + 1)}
m = {l:(l - 1) for l in range(1, 4 + 1)}

# Universal Functions
def euclid_dist(pt1, pt2):
    d = np.linalg.norm(pt1 - pt2)
    return 0.00001 if np.isclose(d, 0) else d # To correct for numerical error

def suppl_vertex_angle(x_k, x_kminus2, d_k, d_kminus1):
    cos = ((euclid_dist(x_k, x_kminus2))**2 - d_k**2 - d_kminus1**2) / (2 * d_k * d_kminus1)
    cos = 1 if np.isclose(cos, 1) else cos # To correct for numerical error
    cos = -1 if np.isclose(cos, -1) else cos # To correct for numerical error
    return np.arccos(cos)

def smooth_window(y_buffer, l, i):
    s_i = np.sum(y_buffer[(i - m[l]):(i + 1)])
    if (((s_i / l) % 1) == 0) and (y_buffer[i - m[l]] != y_buffer[i - m[l] - 1]) and (y_buffer[i] != y_buffer[i + 1]):
        v_i = ((((np.sum(y_buffer[(i - b_lo[l]):((i + b_hi[l]) + 1)]) - s_i) / (w[l] - l)) + 0.5) // 1)
        # y_buffer[(i - m[l]):(i + 1)] = v_i
        for k in range(i - m[l], i):
            print("Converted:"  + str(k) + " is " + str(v_i))
            y_buffer[k] = v_i

def tls(x, y):
    def f(B, x):
        return B[0]*x + B[1]
    linear = odr.Model(f)
    data = odr.Data(x, y)
    fitodr = odr.ODR(data, linear, beta0=[0.1,0.1])
    output = fitodr.run()
    return output.beta

# Classes
class LogisticRegressionModel():
    def __init__(self, prob_threshold, coefs):
        self.p = prob_threshold
        self.beta = coefs
        
    def getprobthreshold(self):
        return self.p
        
    def getcoefs(self):
        return self.beta
    
    def predict(self, X):
        return int((self._probability(X) + (1 - self.p)) // 1)
    
    def _probability(self, X):
        return (1 / (1 + np.exp(-np.dot(X, self.beta))))
        
class WaggleDance():
    def __init__(self, locs_dir, filename): 
        self.locs_dir = locs_dir
        self.filename = filename
        self.k = 0
        self.h = metadata[metadata['DanceID'] == filename.replace('WaggleDance_', '')[:5]]['AvgBeeLength_Px'].values[0]
        self.dance = pd.read_csv(locs_dir + filename)[['x', 'y']].to_numpy()
    
    def getcurrentloc(self):
        if (self.k < len(self.dance)):
            x_k = self.dance[self.k]
            self.k += 1
            return x_k
        return np.array([])
        
    def reset(self):
        self.k = 0
        
intercept, dist_coef, angle_coef, interact_coef, threshold, _, _, _ = all_models[all_models['Dataset'] == '10FPSDataset_v8'].to_numpy()[0]
logreg_10fps_model = LogisticRegressionModel(threshold, [intercept, dist_coef, angle_coef, interact_coef])

intercept, dist_coef, angle_coef, interact_coef, threshold, _, _, _ = all_models[all_models['Dataset'] == 'OriginalDataset_v8'].to_numpy()[0]
logreg_orig_model = LogisticRegressionModel(threshold, [intercept, dist_coef, angle_coef, interact_coef])

---